In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import gudhi


import iblofunmatch.inter as ibfm

from datetime import datetime

_tol = 1e-10

output_dir = "output" # Name of directory to communicate with C++ program
data_dir = os.path.join("datasets", "example-JSC")
plots_dir = os.path.join("plots", "example-JSC")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
RandGen = np.random.default_rng(5)
num_points = 200
Z = ibfm.sampled_circle(1.2,1.8,num_points, RandGen)
indices_subset = list(range(int(num_points*0.2)))
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ibfm.plot_Vietoris_Rips_subset(Z, indices_subset, 0, ax)
plt.legend()
plt.savefig(os.path.join(plots_dir, "points_2.png"))

In [ ]:
# Save dataset 
np.savetxt(os.path.join(data_dir, "Z-computation"), Z)
np.savetxt(os.path.join(data_dir, "indices-subset-computation"), indices_subset) 

In [ ]:
before = datetime.now()
matching_data = ibfm.get_IBloFunMatch_subset(None, Z, indices_subset, output_dir, num_it=1, max_rad=-1, points=True)
after = datetime.now()
print(f"Took {(after-before).total_seconds()} seconds")

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,4))
filt_val = 1.3
ibfm.plot_Vietoris_Rips(Z[indices_subset], filt_val, ax[0], labels=False, fontsize=15, color="gray")
ibfm.plot_Vietoris_Rips(Z, filt_val, ax[1], labels=False, fontsize=15, color="gray")
ibfm.plot_Vietoris_Rips(Z, filt_val, ax[2], labels=False, fontsize=15, color="gray")
ax[0].set_xlim(ax[1].get_xlim())
# Depict three cycles generating domain and their respective images
num_cycles = len(matching_data["S_reps_1"])
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["S_reps_1"][i], ax[0], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)
# Depict image circles 
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["S_reps_im_1"][i], ax[1], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)
# Depict matched circles
block_function_1 = matching_data["block_function_1"]
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["X_reps_1"][block_function_1[i]], ax[2], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)

# Set titles
ax[0].set_title("Cycle representatives in $L'$", fontsize=20)
ax[1].set_title("Embedded representatives in $K'$", fontsize=20)
ax[2].set_title("Matched cycle representatives in $K'$", fontsize=20)

# Save figure
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "cycle_reps_matching_2.png"))

In [ ]:
before = datetime.now()
matching_data = ibfm.get_IBloFunMatch_subset(None, Z, indices_subset, output_dir, num_it=0, max_rad=-1, points=True)
after = datetime.now()
print(f"Took {(after-before).total_seconds()} seconds")

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,4))
filt_val = 1.3
ibfm.plot_Vietoris_Rips(Z[indices_subset], filt_val, ax[0], labels=False, fontsize=15, color="gray")
ibfm.plot_Vietoris_Rips(Z, filt_val, ax[1], labels=False, fontsize=15, color="gray")
ibfm.plot_Vietoris_Rips(Z, filt_val, ax[2], labels=False, fontsize=15, color="gray")
ax[0].set_xlim(ax[1].get_xlim())
# Depict three cycles generating domain and their respective images
num_cycles = len(matching_data["S_reps_1"])
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["S_reps_1"][i], ax[0], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)
# Depict image circles 
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["S_reps_im_1"][i], ax[1], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)
# Depict matched circles
block_function_1 = matching_data["block_function_1"]
for i in range(num_cycles):
    ibfm.plot_cycle(Z, matching_data["X_reps_1"][block_function_1[i]], ax[2], color=mpl.colormaps["Set1"](i/(num_cycles+1)), linewidth=4)

# Set titles
ax[0].set_title("Cycle representatives in $L$", fontsize=20)
ax[1].set_title("Embedded representatives in $K$", fontsize=20)
ax[2].set_title("Matched cycle representatives in $K$", fontsize=20)

# Save figure
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "cycle_reps_matching_2_not_collapsed.png"))

# Table comparing edge collapse iterations and number of point samples 

In [ ]:
from tqdm import tqdm 

RandGen = np.random.default_rng(5)
num_collapse_iterations = 2
num_points_list = list(range(100,200, 20))
num_iterations_points = 5
times_array = np.zeros((num_collapse_iterations+1, len(num_points_list), num_iterations_points))
for k in range(num_collapse_iterations+1):
    print(f"Collapsing {k} times")
    for i, num_points in tqdm(enumerate(num_points_list), total=len(num_points_list)):
        for j in range(num_iterations_points):
            Z = ibfm.sampled_circle(1.2,1.8,num_points, RandGen)
            indices_subset = list(range(int(num_points*0.5)))
            before = datetime.now()
            matching_data = ibfm.get_IBloFunMatch_subset(None, Z, indices_subset, output_dir, num_it=k, max_rad=-1, points=True)
            after = datetime.now()
            times_array[k,i,j] = (after-before).total_seconds()
        # end for
    # end for
# end for

In [ ]:
means_times = times_array.mean(axis=2)
fig, ax = plt.subplots(1,2, figsize=(10,4))
colormap = mpl.colormaps["Set1"]
for i in range(num_collapse_iterations+1):
    print(means_times[i])
    ax[0].plot(list(range(means_times.shape[1])), means_times[i], color=colormap(i/(num_collapse_iterations+1)), label=f"{i} collapse it")
    ax[1].semilogy(list(range(means_times.shape[1])), means_times[i], color=colormap(i/(num_collapse_iterations+1)), label=f"{i} collapse it")

ax[0].legend()
ax[1].legend()
ax[0].set(
    xlabel = "Number of points in Z",
    ylabel="Time (seconds)", 
    xticks = list(range(len(num_points_list))), 
    xticklabels=num_points_list
)
ax[1].set(
    xlabel = "Number of points in Z",
    ylabel="Time log(seconds)", 
    xticks = list(range(len(num_points_list))), 
    xticklabels=num_points_list
)
plt.savefig(os.path.join(plots_dir, "computationvisualisation.png"))